In [34]:
## IMPORTS
import pydicom
import glob
from tqdm import tqdm
import cv2
import datetime

import numpy as np
import cupy as cp

import matplotlib.pyplot as plt
%matplotlib qt

import vispy.plot
import vispy.app
%gui qt

na                  = np.asarray
xm = {
    np:             (np.asarray, np.asarray, np.asarray, np),
    cp:             (cp.asarray, cp.asnumpy, cp.asarray, cp),
}  # (da, ha, xa, xp)
xdarray2xp          = lambda xdarray: {np.ndarray: np, cp.ndarray: cp}[type(xdarray)]

AXALL, AXNEW = (slice(None),), (None,)

ceil = lambda x: int(x) + (int(x) != x)

xd, xh, xa, xp = xm[cp]   
proc_dtype              = np.float16

import xmltodict
import gzip
import skimage.metrics

In [10]:
## COMMONS 
na = np.asarray

def pad_patchify(
        input_array: np.ndarray,                                ##(*bdim, *pdim) 
        padded_patch_shape,                                     ##(             pdim)
        pad_shape,                                              ##({shu+shr},   pdim)
        input_crop_shape,                                       ##({shu+shr},   pdim)
        patch_excess_min,                                       ##(             pdim)
        pad_mode="edge",                                        # numpy.pad(mode=)
        tv = None,                                           # str
):
    pdim    = padded_patch_shape.__len__()                      # patch dimensions no
    bdim    = input_array.shape.__len__() - pdim                # batch dimensions no

    ## calc actual lower padding (not being covered by crop margin)
    pad_shl,  pad_shr   = na(pad_shape)                         # shl, shu, shr = shape [span] lower, upper, rearwards
    crop_shl, crop_shr  = na(input_crop_shape)        
    arr2opa_shl         = crop_shl - pad_shl                    # position of opa's i0 relative to array (consider it vec); opa = outer-padded array
    opa2arr_shl         = pad_shl - crop_shl                    # postition of array's i0 relative to opa
    arr_crop_shl        = np.maximum(0, arr2opa_shl)            # reduce input cropping if no enough crop to satisfy padding
    opa_pad_shl         = np.maximum(0, opa2arr_shl)            # pad needing artificial fulfilling as not being covered by cropped image

    ## calc how much patches will be there  
    array_sh            = na(input_array.shape[ bdim:])
    crarr_sh            = array_sh - (crop_shl + crop_shr)      # crarr = cropped_array
    pdp_sh              = na(padded_patch_shape)                # pdp = padded_patch
    upd_sh              = pdp_sh - (pad_shl+pad_shr)            # upd = unpadded[_patch]
    patch_step          = upd_sh                                # step between 0th elements of consecutive patches
    upd_ex_min_sh       = na(patch_excess_min)
    pgrid_sh, pex_rm_sh = np.divmod(crarr_sh, patch_step)
    pex_accept          = pex_rm_sh > upd_ex_min_sh
    pgrid_sh            = pgrid_sh + pex_accept

    ## calc opa upper padding
    opa_sh              = (pgrid_sh * upd_sh) + (pad_shl + pad_shr)
    arr2opa_shu         = arr2opa_shl + opa_sh 
    opa2arr_shu         = opa2arr_shl + array_sh
    arr_crop_shu        = np.minimum(array_sh, arr2opa_shu)
    opa_pad_shu         = np.minimum(opa_sh,   opa2arr_shu)
    opa_pad_shr         = opa_sh - opa_pad_shu

    ## quick checks
    assert np.all(upd_sh > 0)                                   # is patch containing anything beside patch
    assert np.all(upd_ex_min_sh <= upd_sh)                      # is patch_excess_min positive
    assert np.all(pgrid_sh > 0)                                 # is possible to create at least one patch (by input shape, patch shape, patch excess)

    ## opa = input crop & outer-pad
    batch_sh            = na(input_array.shape[:bdim ])
    batch_all           = (slice(None),)    *bdim
    batch_pad           = (0,)              *bdim
    input_crop          = tuple(slice(aclx, acux)   for aclx, acux in zip(arr_crop_shl, arr_crop_shu))
    opa                 = np.pad(
                                        input_array[batch_all + input_crop], 
                            pad_width=  na((
                                            batch_pad + tuple(opa_pad_shl), 
                                            batch_pad + tuple(opa_pad_shr))).T, 
                            mode=       pad_mode) # type: ignore
    
    ## patchify
    patch_array         = np.zeros(())
    pgrid_og            = np.ogrid[tuple(slice(pgsx) for pgsx in pgrid_sh)]
    patch_og            = np.ogrid[tuple(slice(pasx) for pasx in pdp_sh)]
    pg2pa_og            = tuple(
                                (pgogx * pstepx)[(slice(None),) *pdim   + (None,)           *pdim] 
                            +   paogx           [(None,)        *pdim   + (slice(None),)    *pdim] 
                            for     pgogx,      pstepx,     paogx 
                            in zip( pgrid_og,   patch_step, patch_og))
    
    for _ in tqdm((None,), desc=f"{tv} [PBX={pgrid_sh.prod()}]", disable=not tv):
        patch_array     = opa[batch_all + pg2pa_og]

    ## save info about actual shape for merging
    pgrid_shape         = tuple(pgrid_sh)
    pex_crop_shr        = (upd_sh - pex_rm_sh) * pex_accept
    array_shape         = (pgrid_sh * upd_sh) - pex_crop_shr

    return patch_array, pgrid_shape, array_shape

def pad_unpatchify(
        patch_array: np.ndarray,
        padded_patch_shape,
        pad_shape,
        array_shape,
        tv = None
):
    pdim                = padded_patch_shape.__len__()
    bdim                = patch_array.shape.__len__() - (pdim *2)

    ## calc pads
    pdp_sh              = na(padded_patch_shape)
    pad_shl,  pad_shr   = na(pad_shape)
    upd_sh              = pdp_sh - (pad_shl+pad_shr)
    patch_step          = upd_sh

    pad_shu             = pdp_sh - pad_shr
    pgrid_shape         = patch_array.shape[bdim:-pdim]
    pgrid_sh            = na(pgrid_shape)

    ## unpatchify
    batch_shape         = patch_array.shape[:bdim ]
    batch_all           = (slice(None),) *bdim
    unpad_crop          = (slice(None),) *pdim + tuple(slice(pshlx, pshux) for pshlx, pshux in zip(pad_shl, pad_shu))
    pgrid_og            = np.ogrid[tuple(slice(pgsx) for pgsx in pgrid_sh)]
    patch_og            = np.ogrid[tuple(slice(pasx) for pasx in upd_sh)]
    pg2pa_og            = tuple(
                                (pgogx * pstepx)[(slice(None),) *pdim   + (None,)           *pdim] 
                            +   paogx           [(None,)        *pdim   + (slice(None),)    *pdim] 
                            for     pgogx,      pstepx,     paogx 
                            in zip( pgrid_og,   patch_step, patch_og))

    ## merge
    output_array_ex_sh  = pgrid_sh * upd_sh
    output_array_ex     = np.zeros((*batch_shape, *output_array_ex_sh), dtype=patch_array.dtype)

    for _ in tqdm((None,), desc=f"{tv} [PBX={pgrid_sh.prod()}]", disable=not tv):
        output_array_ex[batch_all + pg2pa_og] = patch_array[batch_all + unpad_crop]

    ## discard excess
    output_array_sh     = array_shape
    output_array        = output_array_ex[(...,) + tuple(slice(osx) for osx in output_array_sh)]

    return output_array

def batchify(
        pbatch_iter,                                            ##(bdim*p[g]dim, pdim)
        patch_shape,                                            ##(pdim)
        umem_pool,                                                # bytes / type.nbytes
        umem_poly,                                               # (1, root(pdim, mlt(patch_sh)), root(pdim, mlt(patch_sh)**2), ..., ) | %todo
        tv=None,
        xp=np,
):
    ## batch_size calc
    pdim                = patch_shape.__len__()
    umem_poly           = na(umem_poly)

    if  umem_poly.shape.__len__() == 1:                     # convert linear polynomial to dimensional one
        umem_poly_lin       = umem_poly
        umem_poly_lin_l     = umem_poly_lin.__len__()
        umem_poly_shx       = int(np.ceil((umem_poly_lin_l-1)/pdim) + 1)
        umem_poly_lin_lp    = (umem_poly_shx * pdim) - pdim + 1
        umem_poly_lin       = np.pad(umem_poly_lin, 
                                (0, umem_poly_lin_lp - umem_poly_lin_l), 
                                "constant", constant_values=0) # type: ignore
        umem_poly_map       = np.mgrid[(slice(umem_poly_shx),) *pdim].sum(axis=0)
        umem_poly           = umem_poly_lin[umem_poly_map]
        patch_shape         = (np.prod(patch_shape) **(1/pdim),) *pdim
    assert  umem_poly.shape.__len__() ==  pdim

    umem_deters         = np.multiply.reduce(na(tuple(
                            np.power(   pshx,           pogx) 
                            for         pshx,           pogx 
                            in zip(     patch_shape,    np.ogrid[tuple(slice(ups) for ups in umem_poly.shape)])),
                            dtype=object))
    umem_estim          = (umem_poly * umem_deters).sum()
    batch_size          = int(umem_pool // umem_estim)
    if not batch_size > 0:
        print(umem_pool/umem_estim)
    assert batch_size > 0

    ## batchify iterations
    # iters_nd            = xp.fromiter(pbatch_iter, dtype=int)
    iters_nd            = xp.asarray(list(pbatch_iter))
    iters_l             = iters_nd.__len__()
    iters_gen           = [((bind.__len__(),), tuple(bind.T))
                            for bix in range(0, iters_l, batch_size)
                            for bind in (iters_nd[bix:(bix+batch_size)],)]
    iters_tq            = tqdm(iters_gen, desc=f"{tv} [PBX={batch_size}/{iters_l}]", disable=not tv)
    return iters_tq

na = np.asarray

zero_kenrel = na([
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],

])

# X- Z| Y\
sobel_kernel3_gen = na([
    [[-1, -2, -1], [-2, -4, -2], [-1, -2, -1]],
    [[ 0,  0,  0], [ 0,  0,  0], [ 0,  0,  0]],
    [[+1, +2, +1], [+2, +4, +2], [+1, +2, +1]],
])
sobel_kernel3s = (
    sobel_kernel3_gen.swapaxes(0, 2), #z
    sobel_kernel3_gen.swapaxes(0, 1), #y
    sobel_kernel3_gen, #x
)[::-1] # (iAxis<kernel>, kZ, kY, kX)


def kernel_apply(patch_pd_shape_in, kernel, kernel_center=None):
    kernel_shape        = kernel.shape
    kernel_center       = kernel_center if kernel_center is not None else tuple((kshx-1)//2 for kshx in kernel_shape)
    assert patch_pd_shape_in.__len__() == kernel_shape.__len__()
    assert all(0 <= kcxx and kcxx < kshx for kcxx, kshx in zip(kernel_center, kernel_shape))

    pad_shape_io        = (
        tuple(kcx           for _,   kcx    in zip(kernel_shape, kernel_center)), 
        tuple((ksx-1)-kcx   for ksx, kcx    in zip(kernel_shape, kernel_center)))

    if (pad_shape_in:=None) is not None: 
        raise NotImplementedError
    else:
        kernel_adjust   = (0,) *kernel_shape.__len__()
        pad_shape_op    = ((0,) *kernel_shape.__len__(),)*2

    patch_pd_shape_op      = tuple(
        patch_shx - (pad_shxl + pad_shul)                      
            for patch_shx, pad_shxl, pad_shul in zip(patch_pd_shape_in, *pad_shape_io))    # unpadded preferably
        
    ixt_gen = tuple((
        kernel_ixt,                                                                                                             # kernel_mask
        tuple(slice(kix + kadjx, asx + kix + kadjx) for kix, asx, kadjx in zip(kernel_ixt, patch_pd_shape_op, kernel_adjust)),  # patch_mask
        ) for kernel_ixt in xp.ndindex(kernel_shape) if kernel[kernel_ixt] != 0
    )
    
    return patch_pd_shape_op, pad_shape_op, ixt_gen

def _kernel_apply2_iter(pgrid_shape, dtype, xp, ixt_gen, patch_pd_shape_op):
    patch_op = xp.zeros((*pgrid_shape, *patch_pd_shape_op), dtype=dtype)
    for ixt_gen_op in ixt_gen: yield (*ixt_gen_op, patch_op)

def kernel_apply2(patch_pd_shape_in, kernel, kernel_center=None, dtype=None):
    patch_pd_shape_op, pad_shape_op, ixt_gen = kernel_apply(patch_pd_shape_in, kernel, kernel_center)
    return patch_pd_shape_op, pad_shape_op, lambda pgrid_shape=(): _kernel_apply2_iter(pgrid_shape, dtype, xdarray2xp(kernel), ixt_gen, patch_pd_shape_op)


In [11]:
## GRADIENT > EXEC
kbim                    = 3 # kernel batch dimension
pdim                    = 3
patch_pdshin            = (16, )    *pdim               # patch_padded_shape_in
pad_shape_in            = ((1, )    *pdim,) *2

def fcn_gradient(dsi):
    xd, xh, xa, xp          = xm[cp]  

    kernel_sobel_d          = xd(sobel_kernel3s)

    dspdin, pgrid_shape, amerge_shape   \
                            = pad_patchify(dsi, patch_pdshin, pad_shape_in, ((0,)*3,)*2, (0,)*3, tv="patchify")

    _osh                    = patch_pdshin
    _osh, _psh, iter_sobel  = kernel_apply2(_osh, kernel_sobel_d[0], (1,)*pdim, xp.float32)
    patch_pdshop            = _osh
    pad_shape_op            = _psh
    dspaop_amp_pl              = np.zeros((3, *pgrid_shape, *patch_pdshop), dtype=proc_dtype)  # dtype=dspain.dtype) # % todo: parametrize when input data signed
    dspaop_arc_pl              = np.zeros((3, *pgrid_shape, *patch_pdshop), dtype=proc_dtype)  # dtype=dspain.dtype) # % todo: parametrize when input data signed

    for bl, bix             in batchify(np.ndindex(pgrid_shape), patch_pdshin, 
                                2**31 //2, (*(0,)*pdim, 120), tv="linear kernel batchified seqencing"):
        patch_in_d          = xd(dspdin[bix])

        for kernel_ixt, patch_sxt, op_sobel in iter_sobel((*bl, kbim)): 
            op_sobel[:] +=  patch_in_d[(*AXALL, *AXNEW, *patch_sxt)]   * kernel_sobel_d[(*AXALL, *kernel_ixt, *AXNEW*3)]
        patch_op_amp     = xp.sqrt(((op_sobel[:, [[1,2], [0,2], [0,1]]]) **2).sum(axis=2)).astype(proc_dtype) # ZYX (as normals)
        patch_op_arc     = xp.arctan2(op_sobel[:, [2,0,1]], op_sobel[:, [1,2,0]]).astype(proc_dtype) # ZYX

        dspaop_amp_pl[(*AXALL, *bix,)] = xh(patch_op_amp.swapaxes(1, 0))
        dspaop_arc_pl[(*AXALL, *bix,)] = xh(patch_op_arc.swapaxes(1, 0))


    dsa_amp_pl          = pad_unpatchify(dspaop_amp_pl, patch_pdshop, pad_shape_op, amerge_shape, tv="unpatchify") #= aom[:, 0]
    dsa_arc_pl          = pad_unpatchify(dspaop_arc_pl, patch_pdshop, pad_shape_op, amerge_shape, tv="unpatchify") #= aom[:, 1]
    return dsa_amp_pl, dsa_arc_pl

In [12]:
## CIRCSEGM > INIT
xd, xh, xa, xp          = xm[cp] 

index_dtype = xp.int16
proc_dtype  = xp.float32
radii_dtype = xp.float32
score_dtype = xp.float32
xd, xh, xa, xp = xm[cp]      
ndim=3       

mmclamm = 600
mm_clamp = (mmclamm, mmclamm+1)

## V mm
cc_mask_stddev = 1
cc_mask_threshold = .3
cc_score_cutoff = .35
cc_radii_cutoff = 5.01

ccrrl = list(np.arange(1, 2.01, 1)) + list(np.arange(3, 16.01, 2))
ccrrl = sorted(ccrrl)
assert any(crx < cc_radii_cutoff for crx in ccrrl)

pad_cc_radius = ceil(max(ccrrl) + (cc_mask_stddev*2))
pad_ms_radius = ceil(max(ccrrl)) #//2

# sh_incircsegm = (20, 512, 512, )
innsh = 128

nd_2d2z = (*AXNEW, *AXALL, *AXALL)
nd_rollaxes = tuple(tuple((idim+jdim)%ndim for idim in range(ndim)) for jdim in range(ndim))
nd_z2i = lambda z2i, idim: xdarray2xp(z2i).moveaxis(z2i, nd_rollaxes[0], nd_rollaxes[idim])


In [13]:
## CIRCSEGM > EXEC
xd, xh, xa, xp = xm[cp]   
ndim = 3
# ndim = 0 ## DISABLE
onlyz = True

def fcn_circsegm(dsa_amp_pl, dsa_arc_pl):
    if True:
        sh_incircsegm   = dsa_amp_pl.shape[-ndim:]
        sh_padded_nd    = tuple(tuple((shx+pad_cc_radius*2) if shi!=idim else innsh for shi, shx in enumerate(sh_incircsegm)) for idim in range(ndim))
        sh_unpadd_nd    = tuple(tuple(na(sh_padded_nd[idim]) - (np.arange(ndim)   != idim)*2*pad_cc_radius) for idim in range(ndim))
        sh_mspadd_nd    = tuple(tuple(na(sh_unpadd_nd[idim]) + (np.arange(ndim)   != idim)*2*pad_ms_radius) for idim in range(ndim))
        sh_inppad_nd    = tuple((tuple((0 if idim==jdim else pad_cc_radius) for jdim in range(ndim)),)*2    for idim in range(ndim))
        sh_mspadd_ms    = tuple(tuple((0,)*2 if idim==jdim else (pad_ms_radius,)*2 for jdim in range(ndim)) for idim in range(ndim))
        pad_ms_mxa = xp.mgrid[(slice(-pad_ms_radius, pad_ms_radius+1),)*2].astype(index_dtype)
        pad_ms_amp = xp.sqrt((pad_ms_mxa.astype(radii_dtype)**2).sum(axis=0))
        pad_ms_mask = (pad_ms_amp <= pad_ms_radius) * (pad_ms_amp > 0)
        iter_adjmax_mask_nd = tuple(tuple(
                (nd_z2i(pad_ms_amp[nd_2d2z], idim)[kernel_ixt], patch_sxt) 
                    for kernel_ixt, patch_sxt in kernel_apply(sh_mspadd_nd[idim], nd_z2i(pad_ms_mask[nd_2d2z], idim))[-1])
            for idim in range(ndim))

        pad_cc_mxa = xp.mgrid[(slice(-pad_cc_radius, pad_cc_radius+1),)*2].astype(index_dtype)
        pad_cc_amp = xp.sqrt((pad_cc_mxa.astype(radii_dtype)**2).sum(axis=0))
        pad_cc_arc = xp.arctan2(pad_cc_mxa[1], pad_cc_mxa[0])

    circ_gen_zyx = []
    for idim in range(ndim):
        circ_gen_i = []
        for ixcr, ccrr in enumerate(ccrrl):
            cc_mask_stddev = cc_mask_stddev 
            ccr_rim = ccrr
            cc_divs = int(ccr_rim*2)

            cc_mask = xp.exp(-((pad_cc_amp-ccr_rim)/cc_mask_stddev)**2)
            cc_mask[cc_mask < cc_mask_threshold] = 0
            cc_mask /= (cc_divs)

            cc_mask_nd = nd_z2i(cc_mask[nd_2d2z], idim)

            cc_arc_discr = (2*xp.pi)/cc_divs
            cc_mask_arc_discr = xp.floor((pad_cc_arc/cc_arc_discr))
            cc_mad_nd = nd_z2i(cc_mask_arc_discr[nd_2d2z], idim)
            # cc_mask_arc = ((cc_mask_arc_discr+.5)*cc_arc_discr)

            iter_ccmask_rad = tuple((
                    (cc_discr+.5)*cc_arc_discr,
                    tuple((
                        tuple(kernel_ixt),
                        tuple(slice(kix, asx + kix) for kix, asx in zip(kernel_ixt, sh_unpadd_nd[idim])),
                    ) for kernel_ixt in xa(xp.where((cc_mask_nd!=0)*(cc_mad_nd==cc_discr))).astype(index_dtype).T))
                for cc_discr in range(-cc_divs//2, cc_divs//2+1))
            
            circ_gen_i.append((cc_mask_nd, ccr_rim, iter_ccmask_rad))
        circ_gen_zyx.append(circ_gen_i)
    print("INIT_DONE")


    image_c = dsa_amp_pl
    image_c = np.clip((image_c-mm_clamp[0])/(mm_clamp[1]-mm_clamp[0]), 0, 1)
    image_a = dsa_arc_pl

    if ndim > 0:
        ds_circsegmout = np.zeros((ndim,) + dsa_amp_pl[0].shape, dtype=radii_dtype)
    for idim in range(ndim):
        if onlyz and idim!=0: break

        image_amp, pgrid_shape, amerge_shape = pad_patchify(image_c, sh_padded_nd[idim], sh_inppad_nd[idim], ((0,)*3,)*2, (0,)*3, tv=f"patchify amp {idim}/{ndim}")
        image_arc, pgrid_shape, amerge_shape = pad_patchify(image_a, sh_padded_nd[idim], sh_inppad_nd[idim], ((0,)*3,)*2, (0,)*3, tv=f"patchify arc {idim}/{ndim}")
        ds_circsegmout_i = np.zeros(pgrid_shape + sh_unpadd_nd[idim], dtype=radii_dtype)

        for bl, bix         in batchify(np.ndindex(pgrid_shape), patch_pdshin, 1, (1,0,0,0,0), tv=f"circular segmentation batchified sequencing {idim}/{ndim}"):
            image_ampb  = xd(image_amp[(idim,) + bix][0])
            image_arcb  = xd(image_arc[(idim,) + bix][0])

            circ_score = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
            circ_radii = xp.zeros(sh_unpadd_nd[idim], dtype=radii_dtype)

            for (cc_mask_d, ccr_rim, iter_ccmask_rad,) in circ_gen_zyx[idim]:
                pdop_ccmask_rad = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
                for kernel_arc, iter_ccmask in iter_ccmask_rad:
                    pdop_ccmask_ri = xp.zeros(sh_unpadd_nd[idim], dtype=score_dtype)
                    image_arcr = xp.cos(image_arcb-kernel_arc)**8
                    for kernel_ixt, patch_sxt, in iter_ccmask: 
                        pccm = image_ampb[patch_sxt] * cc_mask_d[kernel_ixt] * image_arcr[patch_sxt]
                        pdop_ccmask_ri = xp.maximum(pdop_ccmask_ri, pccm)
                    pdop_ccmask_rad += pdop_ccmask_ri

                circ_score_recalc = pdop_ccmask_rad
                ccovsc_bxa = (circ_score < circ_score_recalc)
                circ_radii = np.where(ccovsc_bxa, ccr_rim,              circ_radii)
                circ_score = np.where(ccovsc_bxa, circ_score_recalc,    circ_score)

            circ_score_pd = xp.pad(circ_score, sh_mspadd_ms[idim], constant_values=0)
            circ_radii_pd = xp.pad(circ_radii, sh_mspadd_ms[idim], constant_values=0)

            ccscore_redux =  (cc_radii_cutoff <= circ_radii) * (cc_score_cutoff <= circ_score)
            ccscore_redux_pd = xp.pad(ccscore_redux, sh_mspadd_ms[idim], constant_values=0)

            ms_radii = circ_radii * ccscore_redux
            for kernel_rad, patch_sxt in iter_adjmax_mask_nd[idim]: 
                ms_radii += (ccscore_redux_pd[patch_sxt] 
                    # * (circ_radii_pd[patch_sxt] + kernel_rad > circ_radii) 
                    * (1/(2*xp.pi*kernel_rad))
                    * circ_score_pd[patch_sxt]
                )
            ms_radii_pd = xp.pad(ms_radii, sh_mspadd_ms[idim], constant_values=0)

            circ_filteredout = ms_radii * ccscore_redux
            for kernel_rad, patch_sxt in iter_adjmax_mask_nd[idim]: 
                # circ_filteredout *= (
                #     (ms_radii_pd[patch_sxt] <= ms_radii) # ms circ bigger
                #     + (circ_radii_pd[patch_sxt] <= kernel_rad))## #ms circ or lesser and out of range
                circ_filteredout *= (False
                    + (ms_radii_pd[patch_sxt]*(circ_score_pd[patch_sxt]-cc_score_cutoff) <= ms_radii * (circ_score-cc_score_cutoff)) #
                    + (circ_radii_pd[patch_sxt] <= kernel_rad)## #ms circ or lesser and out of range
        )

            ds_circsegmout_i[bix] = xh(circ_filteredout) #[crop_ms[idim]]
        ds_circsegmout[idim] = pad_unpatchify(ds_circsegmout_i, sh_unpadd_nd[idim], ((0,)*3,)*2, amerge_shape, tv=f"output unpatchify {idim}/{ndim}")

    try:
        del image_c, image_a, 
        del image_amp, image_arc, 
        del image_ampb, image_arcb, 
        del pdop_ccmask_ri, pccm, pdop_ccmask_rad, 
        del circ_score, circ_score_pd, 
        del circ_score_recalc, ccovsc_bxa
        del circ_radii, circ_radii_pd, 
        del ccscore_redux, ccscore_redux_pd
        del ms_radii, ms_radii_pd, circ_filteredout
        del ds_circsegmout_i,
    except: pass
    
    return ds_circsegmout

In [14]:
## VERTSEGM > CLASSIFY SEGMENTS
ndim = 3
idim = 0

junct_split_step = 20
# dscs_db = xd(ds_circsegmout[idim, :, 200:300, 200:300])
index_dtype = xp.int32

def f1d_xa(xa_l):
    lp = xdarray2xp(xa_l[0])
    xa_ = lp.array(xa_l, dtype=object)
    if xa_.shape.__len__() > 1:
        xa = lp.zeros(xa_.shape[0], dtype=object)
        for i in range(xa_.shape[0]): xa[i] = xa_[i]
        return xa
    else: return xa_

index2_dtype = xp.int64

px, py = ((idim+1)%ndim,), ((idim+2)%ndim,)
radconnsq = lambda rm, rx: (xp.maximum(rm, rx)+1)**2 #+(xp.minimum(rm, rx)*0)#((rm+rx)/2)**2 #xp.minimum(rm, rx)**2 #(rm**2)+(rx**2) #xp.maximum(rm, rx)**2 ##


def fcn_vertsegm(ds_circsegmout):
    dscs_db = xd(ds_circsegmout[idim, :])

    dscs_where = xp.where(dscs_db)
    dscs_radii = dscs_db[dscs_where]
    dscs_coord = xa(dscs_where).T

    dscs_junct = xp.zeros(dscs_coord.shape[0], dtype=bool)
    dscs_segme = xp.zeros(dscs_coord.shape[0], dtype=bool)
    dscs_point = xp.zeros(dscs_coord.shape[0], dtype=bool)

    dscs_sljcl = xp.zeros(dscs_coord.shape[0], dtype=index_dtype) -1
    dscs_sljcu = xp.zeros(dscs_coord.shape[0], dtype=index_dtype) -1
    dscs_segml = xp.zeros(dscs_coord.shape[0], dtype=index_dtype) -1

    segm_origs_direct = xp.zeros((0, 2), dtype=index_dtype)

    dl_axa, dl_coord, dl_radii = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T
    dm_axa, dm_coord, dm_radii = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T
    for idx in range(dscs_db.shape[idim]+1):  # dl -> dm -> du; classify junctions and segment elements 
        du_axa      = xp.where(dscs_coord[:, idim]==idx)[0]
        du_coord    = dscs_coord[du_axa]
        du_radii    = dscs_radii[du_axa][(*AXNEW,)].T
        connl = ((dm_coord[:, px] - dl_coord[:, px].T)**2 + (dm_coord[:, py] - dl_coord[:, py].T)**2) <= radconnsq(dm_radii, dl_radii.T)
        connu = ((dm_coord[:, px] - du_coord[:, px].T)**2 + (dm_coord[:, py] - du_coord[:, py].T)**2) <= radconnsq(dm_radii, du_radii.T)
        connl_msum = connl[..., None].sum(axis=1).squeeze()
        connu_msum = connu[..., None].sum(axis=1).squeeze()

        pm_cs_split = not (idx%junct_split_step)
        pm_origin_l = ((connl_msum == 0) * (connu_msum  > 0)) + ((connl_msum == 1) * (connu_msum > 1))
        pm_origin_u = ((connu_msum == 0) * (connl_msum  > 0)) + ((connu_msum == 1) * (connl_msum > 1))
        pm_junct_ex =  (connl_msum  > 1) + (connu_msum  > 1)
        pm_contsegm = ((connl_msum == 1) * (connu_msum == 1))
        pm_point_ex =  (connl_msum == 0) * (connu_msum == 0)

        dscs_junct[dm_axa[pm_origin_l + pm_origin_u + pm_junct_ex + (pm_cs_split * ~pm_point_ex)]] = True
        dscs_point[dm_axa[pm_point_ex]] = True
        dscs_segme[dm_axa[pm_contsegm * (not pm_cs_split)]] = True

        dl_axa, dl_coord, dl_radii = dm_axa, dm_coord, dm_radii
        dm_axa, dm_coord, dm_radii = du_axa, du_coord, du_radii

    dl_axa, dl_coord, dl_radii, dl_junct, dl_segme = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T, xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T , xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T 
    dm_axa, dm_coord, dm_radii, dm_junct, dm_segme = xp.zeros((0,), dtype=index_dtype), xp.zeros((0, ndim), dtype=index_dtype), xp.zeros((0,), dtype=radii_dtype)[(*AXNEW,)].T, xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T , xp.zeros((0,), dtype=index_dtype)[(*AXNEW,)].T 
    for _, idx in enumerate(range(dscs_db.shape[idim]+1)):  # dl -> dm -> du; build segments
        du_axa      = xp.where((dscs_coord[:, idim]==idx))[0]
        du_coord    = dscs_coord[du_axa]
        du_radii    = dscs_radii[du_axa][(*AXNEW,)].T 
        du_junct    = dscs_junct[du_axa][(*AXNEW,)].T 
        du_segme    = dscs_segme[du_axa][(*AXNEW,)].T

        connl = ((dm_coord[:, px] - dl_coord[:, px].T)**2 + (dm_coord[:, py] - dl_coord[:, py].T)**2) <= radconnsq(dm_radii, dl_radii.T)
        connu = ((dm_coord[:, px] - du_coord[:, px].T)**2 + (dm_coord[:, py] - du_coord[:, py].T)**2) <= radconnsq(dm_radii, du_radii.T)
        conn2 = ((dl_coord[:, px] - du_coord[:, px].T)**2 + (dl_coord[:, py] - du_coord[:, py].T)**2) <= radconnsq(dl_radii, du_radii.T)

        sm_juncl_junct, sl_juncl_junct = xp.where(connl * dm_junct * dl_junct.T)
        sl_junc2_junct, su_junc2_junct = xp.where(conn2 * dl_junct * du_junct.T)
        segm_origs_direct = xp.concatenate((segm_origs_direct, 
            xp.vstack((dl_axa[sl_juncl_junct], dm_axa[sm_juncl_junct])).T,
            xp.vstack((dl_axa[sl_junc2_junct], du_axa[su_junc2_junct])).T, 
        ), axis=0)

        sm_segml_junct, sl_segml_junct = xp.where(connl * dm_segme * dl_junct.T) 
        sm_segml_segme, sl_segml_segme = xp.where(connl * dm_segme * dl_segme.T) 
        sm_segmu_junct, su_segmu_junct = xp.where(connu * dm_segme * du_junct.T)  

        dscs_segml[dm_axa[sm_segml_junct]] = dm_axa[sm_segml_junct]             # if new segment started, mark first segment element as self
        dscs_segml[dm_axa[sm_segml_segme]] = dscs_segml[dl_axa[sl_segml_segme]] # if segment is continued, mark continuing element as first one

        dscs_sljcl[dm_axa[sm_segml_junct]] = dl_axa[sl_segml_junct]
        dscs_sljcu[dscs_segml[dm_axa[sm_segmu_junct]]] = du_axa[su_segmu_junct]

        dl_axa, dl_coord, dl_radii, dl_junct, dl_segme = dm_axa, dm_coord, dm_radii, dm_junct, dm_segme
        dm_axa, dm_coord, dm_radii, dm_junct, dm_segme = du_axa, du_coord, du_radii, du_junct, du_segme


    ## VERTSEGM > CONNECT SEGMENTS
    ## .1 > DEFINE SEGMENTS
    segm_markers = dscs_segml[dscs_segme]
    segm_uniqs_markers = xp.unique(segm_markers)
    segm_uniqs = xp.stack((
            dscs_sljcl[segm_uniqs_markers], 
            dscs_sljcu[segm_uniqs_markers],
        ), axis=1)

    e2e_enc = lambda e2e: (e2e[:, 0].astype(index2_dtype) * segm_uniqs.shape[0]) + e2e[:, 1]

    segm_u1_enc = e2e_enc(segm_uniqs)
    segm_u2_enc, segm_u12u2 = xp.unique(segm_u1_enc, return_index=True)
    segm_uniqs = segm_uniqs[segm_u12u2] # reduce parallel segments of the same e2e to first occurence

    dscs_segme_where = xp.where(dscs_segme)[0]
    segm2points = [xp.concatenate((dscs_sljcl[((marker,),)], dscs_segme_where[xp.where(segm_markers == marker)[0]])) for marker in segm_uniqs_markers[segm_u12u2]]

    assert xh(segm_uniqs[:, 0] < segm_uniqs[:, 1]).all()

    # ## .2 > CONNECT JUNCTS
    if True:
        segm_origs_direct = segm_origs_direct
        segm_uniqs_direct = segm_origs_direct

        segm_bfl_direct = xp.zeros((0, 2), dtype=index_dtype)
        segm_bfl_dir_full = []

        segm_build = segm_uniqs_direct[xp.isin(segm_uniqs_direct[:, 0], segm_uniqs[:, 1])].reshape((-1, 1, 2)) # starts only where any uniqs ends
        while True:
            segm_furth = segm_uniqs_direct[xp.isin(segm_uniqs_direct[:, 0], segm_build[:, -1, 1])]
            if segm_furth.shape[0] == 0: break

            segm_bfi_furthb = xp.isin(segm_build[:, -1, 1], segm_furth[:, 0])  # is there any further to be connected
            segm_bfi_accept = xp.isin(segm_build[:, -1, 1], segm_uniqs[:, 0])  # is the end of direct segment start of another standard one

            segm_bfl_direct = xp.concatenate((segm_bfl_direct, segm_build[segm_bfi_accept][:, [0, -1], [0, 1]]))
            segm_bfl_dir_full.extend(segm_build[segm_bfi_accept])

            wh_build, wh_furth = xp.where(segm_build[:, -1, 1][:, None] == segm_furth[:, 0][None, :])
            segm_build = xp.concatenate((segm_build[wh_build], segm_furth[wh_furth].reshape(-1, 1, 2)), axis=1)

            _, segm_ll_e2e = xp.unique(e2e_enc(segm_build[:, (0, -1), (0, 1)]), return_index=True)
            segm_build = segm_build[segm_ll_e2e]
            
        segm_conns_direct, sconnd_wh = (xd(xx) for xx in np.unique(xh(segm_bfl_direct), return_index=True, axis=0))
        segm2points_direct = [segm_bfl_dir_full[xh(ix)][:, 0] for ix in sconnd_wh]
        assert xh(xp.isin(segm_conns_direct[:, 0], segm_uniqs[:, 1]) * xp.isin(segm_conns_direct[:, 1], segm_uniqs[:, 0])).all()

    # ## > MERGE
    print(segm_uniqs.shape, segm_conns_direct.shape)
    print(segm2points.__len__(), segm2points_direct.__len__())
    segm_uniqs = xp.concatenate((segm_uniqs, segm_conns_direct))
    segm2points = segm2points + segm2points_direct  # preserve indices

    assert xh(dscs_coord[segm_uniqs[:, 0]][:, idim] < dscs_coord[segm_uniqs[:, 1]][:, idim])[0]
    assert segm_uniqs.__len__() < np.iinfo(index_dtype).max

    segm_bfl = []
    segm_al_e2e = xp.zeros((0, 2), dtype=index2_dtype)
    segm_efl = xp.zeros((0, 2), dtype=index_dtype)

    # segm_build_total = segm_uniqs[~xp.isin(segm_uniqs[:, 0], segm_uniqs[:, 1])].reshape((-1, 1, 2))
    segm_build_total = xp.where(~xp.isin(segm_uniqs[:, 0], segm_uniqs[:, 1]))[0].reshape((-1, 1))
    segm_build_chunk_len = segm_build_total.__len__() # PARAM

    for bixsc in range(0, segm_build_total.__len__(), segm_build_chunk_len):
        segm_build = segm_build_total[bixsc:(bixsc+segm_build_chunk_len)]
        while True:
            segm_furth = xp.where(xp.isin(segm_uniqs[:, 0], segm_uniqs[segm_build][:, -1, 1]))[0]
            if segm_furth.shape[0] == 0: break

            segm_build_e2e_arl = xp.isin(e2e_enc(segm_uniqs[segm_build][:, (0, -1), (0, 1)]), e2e_enc(segm_al_e2e)) # if started segment already happened to be here
            segm_build_tbcontd = xp.isin(segm_uniqs[segm_build][:, -1, 1], segm_uniqs[segm_furth][:, 0]) # if any more segments to add

            segm_build_fin = segm_build[~segm_build_tbcontd]
            segm_bfl.extend(segm_build_fin)
            segm_efl = xp.concatenate((segm_efl, segm_uniqs[segm_build_fin][:, (0, -1), (0, 1)]))
            print(segm_build_fin.shape, segm_furth.shape, end="; ")
            segm_build = segm_build[segm_build_tbcontd * (~segm_build_e2e_arl)]

            wh_build, wh_furth = xp.where(segm_uniqs[segm_build][:, -1, 1][:, None] == segm_uniqs[segm_furth][:, 0][None, :])
            segm_al_e2e = xp.concatenate((segm_al_e2e, segm_uniqs[segm_build][:, (0, -1), (0, 1)]))
            segm_build = xp.concatenate((segm_build[wh_build], segm_furth[wh_furth].reshape(-1, 1)), axis=1)

            _, segm_ll_e2e = xp.unique(e2e_enc(segm_uniqs[segm_build][:, (0, -1), (0, 1)]), return_index=True)
            segm_build = segm_build[segm_ll_e2e]

    ## VERTSEGM > CONVERT
    segm2points = np.fromiter(segm2points, dtype=object)
    chain2sarg = np.fromiter(segm_bfl, dtype=object)   # segment argument
    chain2se2e = np.fromiter([segm_uniqs[sarg] for sarg in segm_bfl], dtype=object) # end to end of segments by point argument
    chain2ce2e = segm_efl
    # s2p = np.fromiter(segm2points, dtype=object)
    # chain2parg = [xp.concatenate(s2p[xh(sarg)]) for sarg in chain2sarg] # point argument

    print(segm2points.shape, chain2sarg.shape, chain2ce2e.shape, xp.unique(e2e_enc(chain2ce2e)).shape)

    return (
        segm2points, chain2sarg, chain2se2e, chain2ce2e, 
        dscs_coord, dscs_radii, 
        segm_uniqs, segm2points,
        dscs_segme, dscs_junct, dscs_point,
    )

In [15]:
## FILTER > PLANE POINTS
# zkmp = gz
zlen = 1

def ds1_get(zkmp, vertsegm_opt):
    (
        segm2points, chain2sarg, chain2se2e, chain2ce2e, 
        dscs_coord, dscs_radii, 
        segm_uniqs, segm2points,
        dscs_segme, dscs_junct, dscs_point,
    )                           = vertsegm_opt

    # filter long chains, get segments crossing defined plane
    ch_long_enough = xp.where((dscs_coord[chain2ce2e[:, 1]][:, idim] - dscs_coord[chain2ce2e[:, 0]][:, idim]) >= zlen)[0]

    ch_leal_sarg = xp.concatenate(chain2sarg[xh(ch_long_enough)])
    ch_leuq_sarg = xp.unique(ch_leal_sarg)
    clzsl = dscs_coord[segm_uniqs[ch_leuq_sarg]][:, :, 0].T - zkmp # chain long enough unique z kmeans points segment arg localized
    ch_leuq_zkmp = ch_leuq_sarg[(clzsl[0]==0)|(clzsl[1]==0)|((clzsl[0]<0)&(clzsl[1]>0))]
    print(ch_leal_sarg.shape, ch_leuq_sarg.shape, ch_leuq_zkmp.shape, zkmp)

    # get points of plane crossing
    chlzcd = dscs_coord[segm_uniqs[ch_leuq_zkmp]]
    zdiff = chlzcd[:, 1, idim] - chlzcd[:, 0, idim]
    lweight = ((chlzcd[:, 1, idim]-zkmp)/zdiff)[:, None]
    xy_mn = (chlzcd[:, 1, px+py] * (1-lweight)) + (chlzcd[:, 0, px+py] * lweight)

    chlzrad = dscs_radii[segm_uniqs[ch_leuq_zkmp]]
    r_mn = (chlzrad[:, [1]] * (1-lweight)) + (chlzrad[:, [0]] * lweight)

    return xy_mn, r_mn

# fig, ax = plt.subplots(1,1)
# fig.tight_layout()
# ax.imshow(ds0 + 127*ds1c)

# xy_mn.shape, xy_mno.shape

In [16]:
## DATA READ - BASIC
si = 12

tdsp = r"../Training_dataset/{si}/*.dcm".format(si=si)

ds_path_l = glob.glob(tdsp)
ds_path_l = sorted(ds_path_l)

ds_l = [pydicom.dcmread(ds_path) for ds_path in ds_path_l]
ds_ = ds_l[0]
dsa_dstsc = na(ds_.PixelSpacing) / na([0.416015625, 0.416015625])
dsa_dstsh = (na(ds_.pixel_array.shape) * dsa_dstsc).astype(int)

dsa = na([cv2.resize(ds.pixel_array, dsa_dstsh, interpolation=cv2.INTER_CUBIC) for ds in ds_l])
dsi = dsa.astype(proc_dtype)
dsv = (dsa*(256/dsa.max())).astype(np.uint8)


In [17]:
tgot = r"../Training_dataset/{si}/*.QVS".format(si=si)
kk = "Outer Wall/Contour_Point"

dg_path_l = glob.glob(tgot)
gotz_l = []
cpal_l = []

for gzi, dg_path in enumerate(dg_path_l):
    with open(dg_path, "r", errors='ignore') as fh:
        dg = "\n".join(fh.readlines())

    got_raw = xmltodict.parse(dg)["QVAS_Series"]["QVAS_Image"]
    cpal = [
        {
                f"{czi}/{czc['ContourType']}/{pt}": na([(float(po["@x"]), float(po["@y"])) for po in czc[pt]["Point"]]) 
            for czi, czc in enumerate(cz.get("QVAS_Contour", []))
            for pt in ("Contour_Point", "Snake_Point")
        } for cz in got_raw
    ]
    gotz = [(zx, l, got_raw[zx]["@ImageName"]) 
        for zx, l in enumerate([
            len(c.get((
            [key for key in sorted(c.keys()) if kk in key] or [None])[0], [])) for c in cpal]
    ) if l > 0
    ]
    gotz_l.append(gotz)
    cpal_l.append(cpal)



In [19]:
cached = True
dcircsegm_path = "./dcircsegmcache/tr_{si}.npy.gz"

if cached:
    with gzip.open(dcircsegm_path.format(si=si), "rb") as f:
        ds_circsegmout = np.load(f, allow_pickle=True)
else:
    dsa_amp_pl, dsa_arc_pl  = fcn_gradient(dsi)
    ds_circsegmout          = fcn_circsegm(dsa_amp_pl, dsa_arc_pl)
vertsegm_opt                = fcn_vertsegm(ds_circsegmout)
(
    segm2points, chain2sarg, chain2se2e, chain2ce2e, 
    dscs_coord, dscs_radii, 
    segm_uniqs, segm2points,
    dscs_segme, dscs_junct, dscs_point,
)                           = vertsegm_opt


(724, 2) (692, 2)
724 692
(170, 1) (300,); (30, 2) (314,); (56, 3) (374,); (26, 4) (314,); (36, 5) (277,); (32, 6) (187,); (20, 7) (130,); (21, 8) (103,); (20, 9) (74,); (7, 10) (57,); (14, 11) (47,); (6, 12) (41,); (7, 13) (37,); (7, 14) (30,); (5, 15) (26,); (3, 16) (21,); (6, 17) (10,); (1, 18) (6,); (2, 19) (7,); (0, 20) (4,); (1, 21) (5,); (0, 22) (3,); (2, 23) (2,); (0, 24) (1,); (1416,) (472,) (472, 2) (472,)


In [32]:

for gi, gotz in enumerate(gotz_l):
    for gzi, gzs in enumerate(gotz):
        print("    ", gi, gzi, gotz_l[gi][gzi])
        gz = gotz_l[gi][gzi][0]

        ds0 = dsv[gz]
        ds2c = np.zeros(ds0.shape, dtype=np.uint8)
        ds2f = np.zeros(ds0.shape, dtype=np.uint8)

        cpal = cpal_l[gi]
        # for nam in [key for key in cpal[gz].keys() if "Lumen/Contour_Point" in key]: # Lumen/Contour_Point Outer Wall/Contour_Point
        nam = [key for key in cpal[gz].keys() if "Lumen/Contour_Point" in key][0]
        ps = cpal[gz][nam] * dsa_dstsc[None] * (0.845) #- [391-276, 345-207]#* (275/392)#
        cv2.polylines(ds2c, ps.astype(np.int32)[:, ::1][None], 1, 1)
        cv2.fillPoly(ds2f, ps.astype(np.int32)[None], 1)

        xy_mn, r_mn = ds1_get(gz, vertsegm_opt)
        xy_mn_ba = ds2f[tuple(xh(xy_mn.astype(np.int32).T))] > 0
        xy_mno = xh(xy_mn[xy_mn_ba].astype(np.int32))[:, ::-1]  # xy_mno = xh(xy_mn.astype(np.int32))[:, ::-1]
        r_mno = xh(r_mn[xy_mn_ba].astype(np.int32))             # r_mno = xh(r_mn.astype(np.int32))
        print(xy_mn.shape, xy_mno.shape)

        ds1c = np.zeros(ds0.shape, dtype=np.uint8)
        ds1f = np.zeros(ds0.shape, dtype=np.uint8)
        for xy, r in zip(xy_mno, r_mno):
            cv2.circle(ds1c, xy, 1*r.squeeze(), 1, 1)
            cv2.circle(ds1f, xy, 1*r.squeeze(), 1, -1)

fig, ax = plt.subplots(1,1)
fig.tight_layout()
ax.imshow(ds0 + 127*ds2c + 64*ds1c)

     0 0 (136, 67, 'IMG-0012-00137')
(2195,) (773,) (2,) 136
(2, 2) (1, 2)
     0 1 (141, 66, 'IMG-0012-00142')
(2195,) (773,) (3,) 141
(3, 2) (2, 2)
     0 2 (146, 63, 'IMG-0012-00147')
(2195,) (773,) (2,) 146
(2, 2) (1, 2)
     0 3 (151, 65, 'IMG-0012-00152')
(2195,) (773,) (2,) 151
(2, 2) (1, 2)
     0 4 (156, 67, 'IMG-0012-00157')
(2195,) (773,) (2,) 156
(2, 2) (1, 2)
     0 5 (161, 65, 'IMG-0012-00162')
(2195,) (773,) (2,) 161
(2, 2) (1, 2)
     0 6 (166, 64, 'IMG-0012-00167')
(2195,) (773,) (1,) 166
(1, 2) (1, 2)
     0 7 (171, 62, 'IMG-0012-00172')
(2195,) (773,) (2,) 171
(2, 2) (1, 2)
     0 8 (176, 66, 'IMG-0012-00177')
(2195,) (773,) (1,) 176
(1, 2) (1, 2)
     0 9 (181, 85, 'IMG-0012-00182')
(2195,) (773,) (3,) 181
(3, 2) (0, 2)
     0 10 (186, 98, 'IMG-0012-00187')
(2195,) (773,) (4,) 186
(4, 2) (1, 2)
     0 11 (191, 71, 'IMG-0012-00192')
(2195,) (773,) (5,) 191
(5, 2) (1, 2)
     0 12 (196, 67, 'IMG-0012-00197')
(2195,) (773,) (5,) 196
(5, 2) (1, 2)
     0 13 (201, 65, 'I

In [44]:
gi = 1
gzi = 6
gz = gotz_l[gi][gzi][0]

ds0 = dsv[gz]
ds2c = np.zeros(ds0.shape, dtype=np.uint8)
ds2f = np.zeros(ds0.shape, dtype=np.uint8)

cpal = cpal_l[gi]
# for nam in [key for key in cpal[gz].keys() if "Lumen/Contour_Point" in key]: # Lumen/Contour_Point Outer Wall/Contour_Point
nam = [key for key in cpal[gz].keys() if "Lumen/Contour_Point" in key][0]
ps = cpal[gz][nam] * dsa_dstsc[None] * (0.845)
cv2.polylines(ds2c, ps.astype(np.int32)[:, ::1][None], 1, 1)
cv2.fillPoly(ds2f, ps.astype(np.int32)[None], 1)

xy_mn, r_mn = ds1_get(gz, vertsegm_opt)
xy_mn_ba = ds2f[tuple(xh(xy_mn.astype(np.int32).T))] > 0
xy_mno = xh(xy_mn[xy_mn_ba].astype(np.int32))[:, ::-1]  # xy_mno = xh(xy_mn.astype(np.int32))[:, ::-1]
r_mno = xh(r_mn[xy_mn_ba].astype(np.int32))             # r_mno = xh(r_mn.astype(np.int32))
print(xy_mn.shape, xy_mno.shape)

ds1c = np.zeros(ds0.shape, dtype=np.uint8)
ds1f = np.zeros(ds0.shape, dtype=np.uint8)
for xy, r in zip(xy_mno, r_mno):
    cv2.circle(ds1c, xy, 1*r.squeeze(), 1, 1)
    cv2.circle(ds1f, xy, 1*r.squeeze(), 1, -1)

i_pe = (ds1f.sum()/ds0.size)*(ds2f.sum()/ds0.size)

i_tp = ((ds1f & ds2f         )>0).astype(float).sum()        
i_tn = (((~ds1f) & (~ds2f)   )>0).astype(float).sum()    
i_fp = ((ds1f & (~ds2f)      )>0).astype(float).sum()
i_fn = (((~ds1f) & ds2f      )>0).astype(float).sum()

i_accu = (i_tp + i_tn) / ds0.size
i_sens = (i_tp) / (i_tp + i_fn)
i_spec = (i_tn) / (i_tn + i_fp)

i_matt = ((i_tp*i_tn)-(i_fp*i_fn))/np.sqrt((i_tn+i_fn)*(i_fp+i_tp)*(i_tn+i_fp)*(i_fn+i_tp))
i_cohn = (i_accu - i_pe)/(1-i_pe)
i_f1sc = (2*i_tp)/(i_fp+i_fn+(2*i_tp))

i_hdst = skimage.metrics.hausdorff_distance(ds1c, ds2c)

i_moments = cv2.moments(ds1f)
i_coid_xy = [int(i_moments["m10"] / i_moments["m00"]), int(i_moments["m01"] / i_moments["m00"])]
i_tprov = ds1f[tuple(i_coid_xy)[::-1]] > 0
i_fnov  = not i_tprov
i_tpmov = ds2f[tuple(xy_mno[0])[::-1]] > 0 if xy_mno.__len__() > 0 else bool(0)
i_fpov  = not i_tpmov
i_ovlp = (i_tprov + i_tpmov) / (i_tprov + i_tpmov + i_fnov, i_fpov)

print(i_tp , i_tn, i_fp, i_fn, ds0.size)
print(type(i_tn))

print("i_accu", i_accu)
print("i_sens", i_sens)
print("i_spec", i_spec)

print("i_matt", i_matt)
print("i_cohn", i_cohn)
print("i_f1sc", i_f1sc)

print("i_hdst", i_hdst)
print("i_ovlp", i_ovlp)


fig, ax = plt.subplots(1,1)
fig.tight_layout()
ax.imshow(ds0 + 127*ds2c + 64*ds1c)

(2195,) (773,) (2,) 153
(2, 2) (1, 2)
317 91799555.0 0 128 360000
<class 'numpy.float64'>
i_accu 254.99964444444444
i_sens 0.7123595505617978
i_spec 1.0
i_matt 0.844013363217757
i_cohn 254.99992091434294
i_f1sc 0.8320209973753281
i_hdst 3.605551275463989
i_ovlp [ 1. inf]


C:\Users\krism\AppData\Local\Temp\ipykernel_21388\4294175494.py:51: RuntimeWarning: divide by zero encountered in divide
  i_ovlp = (i_tprov + i_tpmov) / (i_tprov + i_tpmov + i_fnov, i_fpov)


array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

(2,)

In [77]:
for si in range(3, 55):
    try:
        print(f"TR = {si}: ", end=" ")
        tdsp = dcircsegm_path.format(si=si)

        ds_path_l = glob.glob(tdsp)
        ds_path_l = sorted(ds_path_l)

        ds_l = [pydicom.dcmread(ds_path) for ds_path in ds_path_l]
        ds_ = ds_l[0]
        dsa_dstsc = na(ds_.PixelSpacing) / na([0.416015625, 0.416015625])
        dsa_dstsh = (na(ds_.pixel_array.shape) * dsa_dstsc).astype(int)

        dsa = na([cv2.resize(ds.pixel_array, dsa_dstsh, interpolation=cv2.INTER_CUBIC) for ds in ds_l])
        dsi = dsa.astype(proc_dtype)
        dsv = (dsa*(256/dsa.max())).astype(np.uint8)

        dsa_amp_pl, dsa_arc_pl  = fcn_gradient(dsi)
        ds_circsegmout          = fcn_circsegm(dsa_amp_pl, dsa_arc_pl)

        with gzip.open(dcircsegm_path.format(si=si), "wb") as f:
            np.save(f, xh(ds_circsegmout), allow_pickle=True)
        print("success!")

    except Exception as e:
        print(e)

TR = 3:  

patchify [PBX=57319]: 100%|██████████| 1/1 [00:01<00:00,  1.57s/it]
linear kernel batchified seqencing [PBX=2184/57319]: 100%|██████████| 27/27 [00:08<00:00,  3.06it/s]
patchify arc 0/3 [PBX=4]: 100%|██████████| 1/1 [00:03<00:00,  3.71s/it]
circular segmentation batchified sequencing 0/3 [PBX=1/4]: 100%|██████████| 4/4 [06:43<00:00, 100.78s/it]
output unpatchify 0/3 [PBX=4]: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


success!
TR = 4:  

patchify [PBX=57319]: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it]
linear kernel batchified seqencing [PBX=2184/57319]: 100%|██████████| 27/27 [00:08<00:00,  3.06it/s]
patchify arc 0/3 [PBX=4]: 100%|██████████| 1/1 [00:03<00:00,  3.82s/it]
circular segmentation batchified sequencing 0/3 [PBX=1/4]: 100%|██████████| 4/4 [06:43<00:00, 100.83s/it]
output unpatchify 0/3 [PBX=4]: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


success!
TR = 5:  

...

success!
TR = 53:  

patchify [PBX=57319]: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it]
linear kernel batchified seqencing [PBX=2184/57319]: 100%|██████████| 27/27 [00:08<00:00,  3.10it/s]
patchify arc 0/3 [PBX=4]: 100%|██████████| 1/1 [00:03<00:00,  3.62s/it]
circular segmentation batchified sequencing 0/3 [PBX=1/4]: 100%|██████████| 4/4 [06:41<00:00, 100.34s/it]
output unpatchify 0/3 [PBX=4]: 100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


success!
TR = 54:  

patchify [PBX=57319]: 100%|██████████| 1/1 [00:01<00:00,  1.60s/it]
linear kernel batchified seqencing [PBX=2184/57319]: 100%|██████████| 27/27 [00:08<00:00,  3.13it/s]
patchify arc 0/3 [PBX=4]: 100%|██████████| 1/1 [00:03<00:00,  3.72s/it]
circular segmentation batchified sequencing 0/3 [PBX=1/4]: 100%|██████████| 4/4 [06:41<00:00, 100.35s/it]
output unpatchify 0/3 [PBX=4]: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


success!


In [17]:
## DRAW > SEGM PIPES
xd, xh, xa, xp = xm[cp]

# ss = xd(np.unique(xh(xp.concatenate([xx for xx in chain2se2e[chacc]])), axis=0)) ## DUMP
# ss = xp.unique(xp.concatenate([xx for xx in chain2sarg[chacc]]))
ss = xp.unique(xp.concatenate([xx for xx in chain2sarg])) ## MOD
ssc = dscs_coord[segm_uniqs[ss]]
ssr = dscs_radii[segm_uniqs[ss]]
print(ss.shape, ssc.shape, ssr.shape)

def drawz(shape, s12l, r12l, val):
    oxt = xp.ogrid[tuple(slice(shx) for shx in shape)]
    out = xp.zeros(shape, proc_dtype)
    for (s1, s2), (r1, r2) in zip(s12l, r12l):
        if s1[0] == s2[0]: continue
        if s1[0]  > s2[0]: s1, s2 = s2, s1
        s2[0] += 1
        vec = s2 - s1
        vec = vec / vec[0]
        rad = r1 + (xp.arange(s2[0]-s1[0]) * (r2-r1)/(s2[0]-s1[0]))

        out[s1[0]:s2[0]] = xp.maximum(out[s1[0]:s2[0]], (
                    ((oxt[1] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[1] + s1[1]))**2) 
                +   ((oxt[2] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[2] + s1[2]))**2) 
            < rad[:, None, None]**2))
    return out * val

def arccz(shape, s12l, r12l, val):
    arccut_tol = xp.pi* .25
    arccut_tol_cos = xp.cos(arccut_tol)

    oxt = xp.ogrid[tuple(slice(shx) for shx in shape)]
    out = xp.ones(shape, proc_dtype)
    for (s1, s2), (r1, r2) in zip(s12l, r12l):
        if s1[0] == s2[0]: continue
        if s1[0]  > s2[0]: s1, s2 = s2, s1
        s2[0] += 1
        vec = s2 - s1
        vec = vec / vec[0]
        # rad = r1 + (xp.arange(s2[0]-s1[0]) * (r2-r1)/(s2[0]-s1[0]))

        out[s1[0]:s2[0]] = xp.minimum(out[s1[0]:s2[0]], xp.cos(xp.arctan2(
                ((oxt[2] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[2] + s1[2]))),
                ((oxt[1] - ((oxt[0][s1[0]:s2[0]]-s1[0])*vec[1] + s1[1])))
            ) + xp.pi - val) <= arccut_tol_cos)
    return out

# t = xh(drawz(dsa.shape, xd(ssc), xd(ssr), 4096)) + dsa # + xh((ds_circsegmout[0]>0)*4e3)
t = xh(drawz(dsa.shape, xd(ssc), xd(ssr), dsa.max()//4)) + dsa
# dsv = dsa * xh(drawz(dsa.shape, xd(ssc), xd(ssr), 1) + (xp.mgrid[tuple(slice(sx) for sx in dsa.shape)][0] > 200))
# t = xh(arccz(dsa.shape, xd(ssc), xd(ssr), 0)) * dsa


(1101,) (1101, 2, 3) (1101, 2)


In [18]:
## DRAW > 3D
figure = vispy.plot.Fig(bgcolor='k', size=(800, 800), show=True) #
volume = figure[0].volume(t[::-1], texture_format='auto', method='mip') # iso|mip

canvas = volume.canvas

@canvas.events.key_press.connect
def on_key_press(event):
    if event.key.name == 'Q': 
        # timer.stop()
        canvas.close()
    if event.key.name == 'A':
        az = figure[0].camera.get_state()['azimuth'] /180*xp.pi
        volume.set_data((xh(arccz(dsa.shape, xd(ssc), xd(ssr), -az)) * dsa)[::-1])
        canvas.show()

canvas.show()
vispy.app.run()


In [23]:
## DRAW DSCS
xd, xh, xa, xp = xm[cp]
dsd = xh((dsa/dsa.max()*255).astype(xp.uint8))
for i in range(dsd.shape[0]):
    si = dscs_coord[:, 0]==i
    cl = dscs_coord[si][:, [2,1]]
    rl = dscs_radii[si]
    for xy, r in zip(cl, rl):
        cv2.circle(dsd[i], xh(xy), int(xh(r)), 128, 1)
    si_s= dscs_coord[(dscs_coord[:, 0]==i)*dscs_segme][:, [2,1]]
    for xy in si_s: cv2.drawMarker(dsd[i], xh(xy), 0, cv2.MARKER_CROSS)
    si_s= dscs_coord[(dscs_coord[:, 0]==i)*dscs_junct][:, [2,1]]
    for xy in si_s: cv2.drawMarker(dsd[i], xh(xy), 0, cv2.MARKER_SQUARE)
    si_s= dscs_coord[(dscs_coord[:, 0]==i)*dscs_point][:, [2,1]]
    for xy in si_s: cv2.drawMarker(dsd[i], xh(xy), 0, cv2.MARKER_TRIANGLE_UP)
    si_s= dscs_coord[(dscs_coord[:, 0]==i)*(~(dscs_segme+dscs_junct+dscs_point))][:, [2,1]]
    for xy in si_s: cv2.drawMarker(dsd[i], xh(xy), 0, cv2.MARKER_DIAMOND)


In [24]:
## VIS -> DCA
# from numba import jit, njit

# %matplotlib ipympl
%matplotlib qt
plt.ion()

view_dsa_l = [
    # dsa+(xh(ds_circsegmout[0]>0) * 1e4),
    # dsa+(xh(ds_circsegmout[1]>0) * 1e4),
    # dsa+(xh(ds_circsegmout[2]>0) * 1e4),
    # ds_circsegmout[0] #, 100:400, 100:400, 100:400]
    t,
    dsd,
]
view_center_axes_names = "ZYX"
view_slice_axes = ((1, 2), (0, 2), (0, 1))  # common for v_dsa
view_center_l = [np.asarray(dsax.shape) //2 for dsax in view_dsa_l]

figsize = 5
vfig_res = (view_dsa_l.__len__(), view_slice_axes.__len__())
vfig, vax_l = plt.subplots(
    *vfig_res,
    figsize=[figsize*vfr for vfr in vfig_res[::-1]])
if vax_l.shape.__len__() == 1:
    vax_l = vax_l[None, ...]

vai_l = np.zeros(vax_l.shape, dtype=list)
for vdsa_ix, vdsa in enumerate(view_dsa_l):
    for vsx_ix, vsxx in enumerate(view_slice_axes):
        vaxx = vax_l[vdsa_ix][vsx_ix]
        blank_im = np.zeros(
            (vdsa.shape[vsxx[0]], vdsa.shape[vsxx[1]]),
            dtype = vdsa.dtype)
        blank_im[0, 0] = vdsa.max()
        axis_image = vaxx.imshow(blank_im)#, cmap='gray')
        axis_center = vaxx.plot([0], [0], marker='+', c='r')
        axis_title = vaxx.set_title("")
        axis_ylabel = vaxx.set_ylabel("")
        
        vai_l[vdsa_ix][vsx_ix] = [
            axis_image,
            axis_center[0],
            axis_title,
            axis_ylabel,
        ]

# @jit(nopython=False)
def update_dsa_view(vdsa_ix):
    vai = vai_l[vdsa_ix]
    vsx = view_slice_axes
    vce = view_center_l[vdsa_ix]
    vdsa = view_dsa_l[vdsa_ix]

    for axex, axes in enumerate(vsx):
        indices = tuple((vce[axix] if axix not in axes else slice(None)) 
            for axix in range(vce.__len__()))
        sliced = vdsa[indices] # if axes[0] < axes[1] else vdsa[indices].T
        # print(vax[axex].imshow(sliced))
        vai[axex][0].set_data(sliced)

        slice_center = ([vce[axes[1]]], [vce[axes[0]]])
        vai[axex][1].set_data(*slice_center)

        indices_str = ", ".join(tuple(f"{axix_name}={vce[axix]}" if axix not in axes else f"{axix_name}:" 
            for axix, axix_name in enumerate(view_center_axes_names)))
        vai[axex][2].set_text(indices_str)

    vai[0][3].set_text(str(vce))
    vfig.canvas.draw()
    # vfig.canvas.flush_events()
    # plt.gcf().canvas.draw()

def update_center(vdsa_ix, vax_ix, xy_center):
    view_center = view_center_l[vdsa_ix]
    slice_axes_updated = view_slice_axes[vax_ix]
    view_center[slice_axes_updated[0]] = xy_center[1]
    view_center[slice_axes_updated[1]] = xy_center[0]

def mouse_callback(event):
    on_canvas = event.inaxes is not None
    xy = (int(event.xdata), int(event.ydata)) if on_canvas else (None, None)
    vdsa_ix, vax_ix = np.argwhere(vax_l == event.inaxes)[0] if on_canvas else (None, None)
    button, dc = (int(event.button), event.dblclick)
    # print(event)
    # print(on_canvas, xy, (vdsa_ix, vax_ix), (button, dc))

    if on_canvas:
        if button == 1:
            update_center(vdsa_ix, vax_ix, xy)
            update_dsa_view(vdsa_ix)
        elif button == 3:
            for vdsa_ix in range(view_dsa_l.__len__()):
                update_center(vdsa_ix, vax_ix, xy)
                update_dsa_view(vdsa_ix)

def keyboard_callback(event):
    k = event.key
    di = 0
    if k == "up": di=-1
    if k == "down": di=1

    for vdsa_ix in range(view_dsa_l.__len__()):
        view_center_l[vdsa_ix][0] += di
        update_dsa_view(vdsa_ix)

for vdsa_ix in range(view_dsa_l.__len__()):
    update_dsa_view(vdsa_ix)

vfig.canvas.mpl_connect('button_press_event', mouse_callback)
vfig.canvas.mpl_connect('key_press_event', keyboard_callback)
vfig.tight_layout()
# vfig.canvas.draw()